In [643]:
import io
import sys
PATH = '/home/epuerta/Academic/AI_DataAnalytics/SLF4SA/'
DIR_DATA = '../SLF4SA/data/input/'
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import preprocessing

from sklearn.pipeline import FeatureUnion
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler


from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, precision_score

from logic.utils import Utils
from logic.classifiers import Classifiers
from logic.text_processing import TextProcessing
from logic.lexical_vectorizer import LexicalVectorizer
from root import DIR_RESULTS



import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, SimpleRNN
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Activation, Dropout, Conv2D

# Variable initialization 

In [644]:
lang = 'es'
iteration = 10
fold = 10
classifiers = Classifiers.dict_classifiers
tp = TextProcessing(lang=lang)
lv = LexicalVectorizer(lang=lang, text_processing=tp)
ut = Utils(lang=lang, text_processing=tp)

# Import data

In [645]:
print('+ Import training...')
x, y = ut.get_data(file_name='tass2020_emotion_train')
print('+ Import test...')
x_eval, y_eval = ut.get_data(file_name='tass2020_emotion_dev')

+ Import training...
		 - Dataset size :(x: 5886 , y: 5886)
+ Import test...
		 - Dataset size :(x: 857 , y: 857)


# Feature extraction

In [646]:
x = lv.transform(x)
x_eval = lv.transform(x_eval)
print('- train:\n', sorted(Counter(y).items()))
print('- eval: \n', sorted(Counter(y_eval).items()))

- train:
 [('anger ', 600), ('disgust ', 113), ('fear ', 67), ('joy ', 1270), ('others', 1), ('others ', 2888), ('sadness ', 706), ('surprise ', 241)]
- eval: 
 [('anger ', 87), ('disgust ', 16), ('fear ', 10), ('joy ', 185), ('others ', 421), ('sadness ', 103), ('surprise ', 35)]


# OVERSAMPLING

In [647]:
k_fold = ShuffleSplit(n_splits=fold, test_size=0.25, random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
ros_train = RandomOverSampler(random_state=1000)
x_train, y_train = ros_train.fit_resample(x_train, y_train)
x_test, y_test = ros_train.fit_resample(x_test, y_test)
print('- train:\n', sorted(Counter(y_train).items()))
print('- test:\n', sorted(Counter(y_test).items()))

- train:
 [('anger ', 2022), ('disgust ', 2022), ('fear ', 2022), ('joy ', 2022), ('others', 2022), ('others ', 2022), ('sadness ', 2022), ('surprise ', 2022)]
- test:
 [('anger ', 866), ('disgust ', 866), ('fear ', 866), ('joy ', 866), ('others ', 866), ('sadness ', 866), ('surprise ', 866)]


In [663]:

shape = x_train.shape[1:], y_train.shape[1:]
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [664]:
lb = LabelEncoder()
y_train = lb.fit_transform([i.rstrip() for i in y_train.to_list()])
y_test = lb.fit_transform([i.rstrip() for i in y_test.to_list()])

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [ ]:
y_train

In [665]:
x_train

array([[-0.03790831,  0.88515136, -0.07960854, ..., -0.1513908 ,
        -0.11821717, -0.29899566],
       [-0.03790831, -0.69503661, -0.75838369, ..., -0.1513908 ,
        -0.11821717, -0.34442369],
       [-0.03790831, -0.69503661, -0.60545807, ..., -0.1513908 ,
        -0.11821717, -0.66502707],
       ...,
       [-0.03790831, -0.69503661, -0.33783822, ..., -0.1513908 ,
        -0.11821717, -0.44648893],
       [-0.03790831, -0.69503661, -0.33783822, ..., -0.1513908 ,
        -0.11821717, -0.44648893],
       [-0.03790831, -0.69503661,  2.60598011, ..., -0.1513908 ,
        -0.11821717,  1.31857617]])

In [666]:
shape

((27,), ())

# RNN Architecture

In [667]:
modelo = Sequential()

In [668]:
modelo.add(TimeDistributed(LSTM(27, activation='relu')))

In [669]:
modelo.add(Dropout(0.2))

In [670]:
modelo.add(LSTM(1024,input_shape =(None, 27), return_sequences=True))

In [671]:
modelo.add(TimeDistributed(Dense(27)))

In [672]:
modelo.add(Activation('softmax'))

In [673]:
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [674]:
#modelo = models.Sequential()

In [675]:
#modelo.add(layers.Conv2D())

In [676]:
history = modelo.fit(x_train, y_train,epochs=50)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\layers\wrappers.py", line 174, in build
        raise ValueError(

    ValueError: Exception encountered when

# Training

In [522]:
modelo.fit(x=x_train, y=y_train, epochs = 1, validation_data = (x_test, y_test))

ValueError: in user code:

    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\CARLOS\anaconda3\lib\site-packages\keras\layers\wrappers.py", line 174, in build
        raise ValueError(

    ValueError: Exception encountered when

# Result

In [303]:
pd.DataFrame(history.history).plot(figsize=(12, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2)
plt.show()

NameError: name 'history' is not defined